# Medical appointment no-shows analysis
This analysis is part of the Udacity Data Analysis Nanodegree program and aims to explore a dataset containing aproximately 100k medical appointments from the Brazilian public health system. It is basically divided into four sections: (1) *Introduction*, where the investigation problem is set and the dataset is described; (2) *Data Wrangling*, where the acquired data are cleaned and parsed; (3) *Exploratory Data Analysis*, in which some intuition about the data are built based on the found patterns; (4) *Conclusion*, where the found insights about the problem are reviewed and communicated. Although the main techniques involved in data analysis are applied, we highlight the use of heatmaps to gain insights regarding the study topic.

Those readers not interested in the whole data analysis process may read <a href="#intro">Section 1</a>, to get familiar with the data and the investigation problem and then jump directly to <a href="#conclusions">Section 4</a> to find out the analysis results.

## Table of Contents
* <a href="#intro">1. Introduction</a>
  * <a href="#sec1.1"> 1.1. Initial statements </a>
  * <a href="#sec1.2"> 1.2. Understanding the data </a>
  * <a href="#sec1.3"> 1.3. Hypothesis to be explored </a>

* <a href="#wrangling">2. Data Wrangling</a>
  * <a href="#sec2.1"> 2.1. General properties </a>
  * <a href="#sec2.2"> 2.2. Numerical attributes and outliers</a>
    * <a href="#sec2.2.1"> PatientID</a>
    * <a href="#sec2.2.2"> AppointmentID</a>   
    * <a href="#sec2.2.3"> Checking the Age attribute</a>   
    * <a href="#sec2.2.4"> Checking the Handcap attribute</a>   
  * <a href="#sec2.3"> 2.3. Categorical attributes</a>
    * <a href="#sec2.3.1"> Gender attribute</a>   
    * <a href="#sec2.3.2"> Schedule and appointment date</a>
    * <a href="#sec2.3.3"> Waiting days</a>
    * <a href="#sec2.3.4"> Neighborhood names</a>
    * <a href="#sec2.3.5"> No-show class</a>
  * <a href="#sec2.4"> 2.4. Data wrangling overview</a>
  
* <a href="#eda">3. Exploratory Data Analysis</a>
  * <a href="#EDA_question1">3.1. Exploring no-showing appointments</a>
    * <a href="#sec3.1.1"> Waiting time between the scheduling and the appointment date </a>
    * <a href="#sec3.1.2"> A return to assiduity </a>
    * <a href="#sec3.1.3"> Neighborhood analysis </a>    
  * <a href="#EDA_question2">3.2. Exploring the patient profiles</a>

* <a href="#conclusions">4. Conclusions</a>
  * <a href="#sec4.1"> 4.1. Analysis limitations</a>
  * <a href="#sec4.2"> 4.2. Analysis overview</a>
* <a href="#references">References</a>

<a id='intro'></a>
## 1. Introduction

The Brazilian public health system, known as SUS for *Unified Health System* in its acronym in Portuguese, is one of the largest health system in the world <a href="#[1]">[1]</a>, representing government investment of more than 9% of GDP. However, its operation is not homogeneous and there are distinct perceptions of quality from citizens in different regions of the country.

This analysis will use the *no-show appointments dataset* which collects information from 100k medical appointments in Brazil and is focused on the question of whether or not patients show up for their appointment, including a set of characteristics about the patient in each row:
- ‘ScheduledDay’: tells us on which day the patient set up their appointment.  
- ‘Neighborhood’: indicates the location of the hospital.  
- ‘Scholarship’: indicates whether or not the patient is enrolled in Brasilian welfare program called *Bolsa Família*.
- 'No-show': it says ‘No’ if the patient showed up to their appointment, and ‘Yes’ if they did not show up.

This analysis aims to outline some possible reasons for patient no-showing at the scheduled appointments, as well as get insights about the Brazilian public health system. To accomplish this, we will first try to understand the data and the context on which they were collected. We will then proceed to data wrangling and exploratory data analysis, in an iterative process, in order to draw some conclusions about the subject.

<a id='sec1.1'></a>
### 1.1. Initial statements
This section sets up import statements for all the packages that will be used throughout this python notebook.

In [ ]:
# Data analysis packages:
import pandas as pd
import numpy as np
#from datetime import datetime as dt

# Visualization packages:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

<a id='sec1.2'></a>
### 1.2. Understanding the data
Since the original dataset is available at Kaggle <a href="#[2]">[2]</a>, the first step was to get additional information available on its original repository, as transcripted below:

>In response here it goes: Country? Brazil. Is this from private or public service? Or both? Public Sector (Brazil). Primary, secondary, or tertiary care? Primary care, not sure about it. Only physicians? What would be the specialties? We don't have these information.
  
>First appointment with that specific professional? I believe first appointments are prone to "no show". That is a very good hint, we don't have that information yet. Part of the day? Early morning might be worse. (Perhaps in the next version we can include it.

Another relevant point is that any information is provided about the source of data. I.e., did these data come from a government agency or were they a compilation from each health unit? Furthermore, besides it is known the data is about public health unit, it is not said from which government sphere they belong to (municipal, state or federal). 
The absence of such information can result in a biased analysis, since the complex and decentralized Brazilian health system brings inevitable conflicts and contradictions between its associated entities, as identified in <a href="#[3]">[3]</a>.

This question regarding the data source was made at the Kaggle discussion forum (available at https://goo.gl/JB4P4d), on which the following answer were given: 
> This is at municipal level combining the appointments occurred in more then 45 units. As far as we know, the city has 20 million loss a year with no-shows. After so many studies happening in this data the average no-show now is 20%. Its important to mention that all services offered to the municipality were free of charge.

In order to define the hypotheses that will lead this analysis, it is first needed to know which information is available in the dataset (attributes). Thus, the first lines of the dataset are shown below:

In [ ]:
## Reading the dataset file name:
import os
print(os.listdir("../input"))

In [ ]:
pd.read_csv('../input/KaggleV2-May-2016.csv').head()

<a id='sec1.3'></a>
### 1.3. Hypotheses to be explored

From the dataset information and its attributes, the following questions can be formulated:
* **Regarding the no-showing appointments**:
  - Do the patients forget the appointment?
  - What is the average waiting time between the scheduling date and the appointment date?
  
* **Regarding the patient profile**:
  - Is there any common characteristics among those patients that miss appointments?
  - What is the *missing appointments per patient* relation in the dataset?  
  - How many patients have missed an appointment at least once?
  - Patients suffering from serious illnesses are more assiduous?

* **Regarding the health unit**:
  - What are the neighbourhoods with the highest no-showing rate?
  - Considering there are common characteristics among those who do not attend the appointments, how these characteristics are geographically distributed?
  
* **Regarding the data**:
  - Is the data balanced in relation to the interest class (showing/no-showing to appointments)?
  - How is the data distributed in relation to the location (neighborhood)? I.e. all the units have an equivalent and balanced number of instances?
  
The analysis conduced in the next sections will try to address some of these questions answering them based on the available dataset.

-------------

<a id='wrangling'></a>
## 2. Data Wrangling
In this section the data will be loaded and some operations will be performed to inspect data types, to look for missing values or possibly errant data. Data cleaning operations will be executed in the same section where a specific attribute is being analyzed.

<a id='sec2.1'></a>
### 2.1. General Properties

In [ ]:
## Loading the dataset and printing out a few lines:
dataset = pd.read_csv('../input/KaggleV2-May-2016.csv')
dataset.head(3)

In [ ]:
## Reading dataset general information:
dataset.info()

From the command above, it is possible to notice the dataset has no missing values, since all the attributes have 110.527 instances. However there is no guarantee these values are all consistent. In the way to check it, the *pandas.describe()* method will be applied in the next section to gather information from the numerical attributes.

<a id='sec2.2'></a>
### 2.2. Numerical attributes and outliers

In [ ]:
## Describing the numerical attributes:
dataset.describe()

<a id='sec2.2.1'></a>
#### 2.2.1 PatientID:
This attribute type is originally numerical *(float64 type)*. However, since it represents the patient identification, any numerical operations should not be applied to it and therefore it will be transformed into a string type:

In [ ]:
## Checking the attribute data type:
type(dataset['PatientId'][0])

In [ ]:
## Converting the values to int type and then to str type:
dataset['PatientId'] = dataset['PatientId'].apply(lambda x: str(int(x)));

In [ ]:
## Counting how many unique patients are in the dataset:
len(dataset['PatientId'].unique())

Counting the unique values of this dataset attribute shows us how many unique patients we are dealing with. In this case, there are 62,299 patients which indicates that some patients had more than one appointment *(an average of 1.77 appointments per patient)*. 

<a id='sec2.2.2'></a>
#### 2.2.2 AppointmentID:
The same can be said for the *AppointmentID* attribute, which will also be transformed into a string type:

In [ ]:
## Checking the attribute data type:
type(dataset['AppointmentID'][0])

In [ ]:
## Converting the values to int type and then to str type:
dataset['AppointmentID'] = dataset['AppointmentID'].apply(lambda x: str(int(x)));

In [ ]:
## Counting how many unique patients are in the dataset:
len(dataset['AppointmentID'].unique())

This attribute seems to be consistent: there are 110,527 instances in the dataset, as well as 110,527 unique values for the appointmentIDs. For last, since each instance corresponds to an appointment register, we will redefine the dataset index to the appointment IDs. 

In [ ]:
dataset.set_index('AppointmentID', drop=True, inplace=True)

<a id='sec2.2.3'></a>
#### 2.2.3. Checking the *Age* attribute:
It is not possible to be aged $-1$, as well as older than $100$ years calls some attention. The latter will be analyzed appropriately at the exploratory data analysis section, but agest under zero are definitely a mistake. In this case, we will first check how many instances correspond to this before treat them.

In [ ]:
dataset[dataset['Age']<0]

Since there is just one occurrence, it will be removed from the dataset.

In [ ]:
dataset.drop('5775010',inplace=True)  #Removing the anomalous instance
# dataset.reset_index(drop=True,inplace=True)  #Reseting the dataset index

<a id='sec2.2.4'></a>
#### 2.2.4. Checking the *Handcap* attribute:
In the dataset metadata it is said that the *handcap* attribute should cointain a boolean value. However, as seen above, this attribute assumes values from 0 to 4, probably indicating the handicap number for each patient. In this analysis, we will map to 1 any value higher than 0.  

Also, there is a typo in the attribute name (it should be *handicap*) which will be corrected later.

In [ ]:
## Converting all 'Handcap' values higher than 0 to 1:
dataset['Handcap'] = np.where(dataset['Handcap']>0, 1, 0)

<a id='sec2.3'></a>
### 2.3. Categorical attributes

In [ ]:
## Getting information of the categorical attributes:
dataset.info()

<a id='sec2.3.1'></a>
####  2.3.1. *Gender* attribute:
Only two values are expected from this attribute. In order to check its consistency, the sum for both **M**ale and **F**emale classes must equal the total number of instances.

In [ ]:
## Counting gender classes
dataset.Gender.value_counts()

Although the attribute data are consistent (the total sums 110,526 instances), they are unbalanced. Since each instance represents the appointment and not the patient, this fact can be treated in a further analysis.

<a id='sec2.3.2'></a>
####  2.3.2. Schedule and appointment date:
These two attributes are related to when the patient scheduled *(ScheduledDay)* and to which date he scheduled the appointment *(AppointmentDay)*. It is effort to notice, though, that the *AppointmentDay* has just the date information, while the *ScheduledDay* has both date and time information.  
In this section we will parse the data in order to find out how many days have passed between the schedule date and the appointment date. To get to it, we could use the python native datetime library. However, we choose to use the pandas methods for datetime processing <a href="#[4]">[4]</a>.

In [ ]:
## Reading again the dataset first lines to get acquainted with its content:
dataset.head(2)

##### Parsing the date and time information:

In [ ]:
## Converting the date information in string to datetime type:
dataset['ScheduledDay'] = pd.to_datetime(dataset.ScheduledDay)
dataset['AppointmentDay'] = pd.to_datetime(dataset.AppointmentDay)
## Creating a new column (attribute) containing just the scheduling time:
dataset['ScheduleTime'] = dataset.ScheduledDay.dt.time
## Normalizing the "Day" columns to keep just the date information (dropping the time info)
dataset['ScheduledDay'] = dataset.ScheduledDay.dt.normalize()

<a id='sec2.3.3'></a>
#### 2.3.3. Waiting days

A relevant information for this analysis is the waiting time, in days, between the scheduling date and the appointment date. To get to this, we will first have to parse the data through the following steps:*
* Create a "Waiting Days" column;
* Categorize these values in the context of this analysis;
* Insert the correspondent categories in the working dataset.

In [ ]:
## Since both 'AppointmentDay' and 'ScheduledDay' are pandas.Timestamp type, this operation can be done directly:
dataset['WaitingDays'] = dataset['AppointmentDay'] - dataset['ScheduledDay']

At this point we must **check the data consistency** in relation to the scheduling and appointment times: if the new *WaitingDays* attribute has negative values, this indicates that the scheduling was performed after the appointment and such record should therefore be handled.  

To help further analysis, a function will be created to return a *time delta* type from a given number of days. 

In [ ]:
def waiting_days(days):
    '''Auxiliary function to parse a date information from string type to python datetime object.
    Syntax: waiting_days(days), where:
        days = int type with the number of days considered.
    Return: a correspondent pandas._libs.tslib.Timedelta data type.
    '''
    arg = str(days) + ' days'
    return pd.tslib.Timedelta(arg)

In [ ]:
## Checking which instances were scheduled after the appointment:
dataset[dataset['WaitingDays'] < waiting_days(0)]

Since few inconsistent instances have been found, they will be removed:

In [ ]:
## Recording the inconsistent instances index 
dropIx = dataset[dataset['WaitingDays'] < waiting_days(0)].index
## Dropping these instances from the dataset:
dataset.drop(dropIx, inplace=True)

The latest operations return a *pandas..Timedelta* type. However, in order to analyze this data, we need first to get just the number of days using a native method from pandas library:

In [ ]:
dataset['WaitingDays'] = dataset.WaitingDays.dt.days  #Extract just the day value from the full "timedelta" object.

Now, making use of another dataframe, these data will be parsed appropriately making use of the *groupBy* method, which returns a *pandas.groupby object* with the selected attributes as index:

In [ ]:
## Grouping by the 'WaitingDays' and 'No_show' values:
waitingdays = dataset.groupby(by=['WaitingDays','No-show'])

Since we are interested in the number of instances grouped either by 'WaitingDays' as by 'No_show' attributes, we will then use the *count()* method. To avoid redundancy we choose the 'PatientId' as reference, but it could be applied in any other attribute. We make use of the *unstack()* method to transform the hierarchical index as dataframe columns again.

In [ ]:
waitingdays = waitingdays.count()['PatientId'].unstack()

Since for a given number of waiting days there is no correspondent values for *No_show: Yes* or *No_show: No*, the last operation will result in a *NumPy.NaN* value. However, in order to be able to plot these data, we will replace those NaN for 0. We will also reset the index, moving its values to a new dataframe column:

In [ ]:
waitingdays.fillna(value=0, inplace=True)
waitingdays.reset_index(drop=False, inplace=True)
waitingdays.info()

The days some patients have to wait to be attended varies from 0 to 128. In order to enable a data analysis, we will categorize them as follow, using the *cut()* method as explained in <a href="#[5]">[5]</a>:

Waiting time   | Number of days
--- | ---
Same day | 0
Short | 1 - 3
Week | 4 - 7
Fortnight | 8 - 15
Month | 16 - 30
Quarter | 31 - 90
Semester | 91 - 180
Very long | > 180

In [ ]:
## Defining the categories label:
categories = pd.Series(['Same day: 0', 'Short: 1-3', 'Week: 4-7', 'Fortnight: 8-15', 'Month: 16-30', 'Quarter: 31-90', 'Semester: 91-180', 'Very long: >180'])

In [ ]:
## Applying these categories both to the auxiliary and to the working datasets:
waitingdays['WaitingDays'] = pd.cut(waitingdays.WaitingDays, bins = [-1,0,3,7,15,30,90,180, 10000], labels=categories)
dataset['WaitingCategories'] = pd.cut(dataset.WaitingDays, bins = [-1,0,3,7,15,30,90,180, 10000], labels=categories)

The categorized waiting days information was updated in the main dataset. However, it would be useful to parse the auxiliary dataset (*waitingdays*) in order to find out how the no-showing rate is distributed among the waiting categories.

In [ ]:
## Grouping the dataset by the waiting categories, returning the sum of all instances:
waitingdays = waitingdays.groupby('WaitingDays').sum()
## Creating a new attribute, "No-showing rate", relating how many patients did not show up against those who did.
waitingdays['No-showing rate'] = (waitingdays.Yes / waitingdays.No)*100

In [ ]:
## Viewing the resulting dataset:
waitingdays

<a id='sec2.3.4'></a>
#### 2.3.4. *Neighborhood* names
Accordingly to the dataset description, this attributes refers to the neighborhoods the health units are located in the city of *Vitória*, in the State of *Espírito Santo*, Brazil.

In [ ]:
## Checking the unique neighborhood names:
neighborhood = dataset.Neighbourhood.unique()
neighborhood.sort()  #Sorting the names in alphabetical order
neighborhood  #Showing the results

Considering "ILHAS OCEÂNICAS DE TRINDADE" is not an official neighborhood of *Vitória* <a href="#[8]">[8]</a>, the two instances related to them will be dropped off. 

In [ ]:
dataset.drop(dataset[dataset['Neighbourhood'] == 'ILHAS OCEÂNICAS DE TRINDADE'].index, inplace=True)

In [ ]:
## Counting again the neighborhood number:
neighborhood = dataset.Neighbourhood.unique()
neighborhood.sort()

In [ ]:
## Counting neighborhood:
len(neighborhood)

In [ ]:
## Plotting an histogram with the neighborhoods sorted alphabetically. 
plt.figure(figsize=(16,6))
ax = sns.countplot(x='Neighbourhood', data=dataset, order=neighborhood)
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right", fontsize=8)
plt.title('Distribution of appointments per neighborhood', fontsize=14, fontweight='bold')
plt.show()

From the graph above, it can be noticed there are 81 different neighborhood being considered, where 8 of them register more than 3k appointments. However, in the data description (see <a href="#intro">Section 1</a></li>) it is said there are 45 health units, which leads us to consider the neighborhood as where the patient lives.  

Due to this, EDA involving the neighborhood must be carried out in a careful manner.

<a id='sec2.3.5'></a>
####  2.3.5. *No-show* class:
We want to check wheter the class values are consistent with the expected values for this database (true or false).

In [ ]:
## Counting gender classes
dataset['No-show'].value_counts()

The total sum is consistent with the dataset length. However, as it was also noticed for the *Gender* attribute, this data are unbalanced, i.e. there are more instances pertaining to one class than other. Since this attribute represents our dataset categorization, this fact becomes more relevant for the further analysis. In other words, although we can get some useful insights from this data, we must be careful when applying this dataset to training machine learning algorithms and so on. 

<a id='sec2.4'></a>
### 2.4. Data wrangling overview

Just in order to help its reading, the dataset attributes will be rearranged to put all the date and time information together, as well as correcting the typos in the attribute names:

In [ ]:
## Reading the dataset attributes (columns):
dataset.columns

In [ ]:
dataset = dataset.reindex(columns=['PatientId', 'Gender', 'Age', 'Scholarship', 'Hipertension', 'Diabetes',
       'Alcoholism', 'Handcap', 'ScheduledDay', 'ScheduleTime', 'AppointmentDay', 'WaitingDays', 'WaitingCategories', 'SMS_received', 
       'Neighbourhood', 'No-show'])

In [ ]:
## Reading again the current attribute labels:
dataset.columns

In [ ]:
## Renaming "No-show"to "No_show"; "Handcap" to "Handicap"; and "ScheduleTime" to "ScheduledTime":
dataset.columns = ['PatientId', 'Gender', 'Age', 'Scholarship', 'Hipertension', 'Diabetes',
       'Alcoholism', 'Handicap', 'ScheduledDay', 'ScheduledTime', 'AppointmentDay', 'WaitingDays', 
       'WaitingCategories', 'SMS_received', 'Neighbourhood', 'No_show']

We then conclude this data wrangling step by showing an overview of the pre-processed data, i.e. the data after the wrangling and cleansing process. 

In [ ]:
## Checking again the dataset information (for numerical attributes) and description (for categorical ones):
print(dataset.info())
dataset.describe()

In [ ]:
## Visualizing few instances of the data:
dataset.head(3)

As it can be seen, the numerical attributes are consistent with the expected values (i.e. counting 110.519 values, with a minimum of 0 and maximum value of 1 to the boolean attributes, while in a range from 0 to 115 for the age attribute). In the same way, the categorical attributes are as expected.

-------------

<a id='eda'></a>
## 3. Exploratory Data Analysis

Once the data were trimmed and cleaned, we will move on to explore the questions posed on  <a href="#sec1.3">Section 1.3</a></li>. Besides it was identified in <a href="#wrangling">Section 2</a></li> that data are unbalanced, it will not be taken into account in the following analysis. Given the dataset limitations, this analysis will address the first two questions, as organized in the following topics::  

* **<a href="#EDA_question1">3.1. Exploring no-showing appointments</a>**
* **<a href="#EDA_question2">3.2. Exploring the patient profiles</a>**

Some functions will be defined below in order to facilitate the further analyzing process.

In [ ]:
def get_statistics(data, bins=20):
    '''Prints basic statistics from the input data. 
    Syntax: get_statistics(data, bins=20), where:
        data = the input data series;
        bins = the number of bins to the histogram.
    '''
    total = data.values
    print('Mean:', np.mean(total))
    print('Standard deviation:', np.std(total))
    print('Minimum:', np.min(total))
    print('Maximum:', np.max(total))
    print('Median:', np.median(total))
    plt.hist(data, bins=bins);

In [ ]:
def get_total(dataframe):
    '''Return the total sum of each numerical attribute of a pandas.Dataframe.'''
    return dataframe.sum(axis=1)

In [ ]:
def df_row_normalize(dataframe):
    '''Normalizes the values of a given pandas.Dataframe by the total sum of each line.
    Algorithm based on https://stackoverflow.com/questions/26537878/pandas-sum-across-columns-and-divide-each-cell-from-that-value'''
    return dataframe.div(dataframe.sum(axis=1), axis=0)

In [ ]:
def df_column_normalize(dataframe, percent=False):
    '''Normalizes the values of a given pandas.Dataframe by the total sum of each column.
    If percent=True, multiplies the final value by 100.
    Algorithm based on https://stackoverflow.com/questions/26537878/pandas-sum-across-columns-and-divide-each-cell-from-that-value'''
    if percent:
        return dataframe.div(dataframe.sum(axis=0), axis=1)*100
    else:
        return dataframe.div(dataframe.sum(axis=0), axis=1)

<a id='EDA_question1'></a>
### 3.1. Exploring the *no-showing appointments*
In this section we will seek to answer the following questions based on the available data:  
* What is the average waiting time between the scheduling date and the appointment date?
* Is there any relation between the waiting time and the no-showing appointments?
* How the *no-showing appointments* are distributed among the health units (neighborhood)?

In orther to find the answers, we will now focus on those attributes related specifically to appointments: *patient ID; scheduled date; appointment date; and health unit neighborhood*, as well as those attributes resulting from these.

<a id='sec3.1.1'></a>
#### 3.1.1. Waiting time between the scheduling and the appointment date
To get an overall idea of the no-showing appointments, we will start analyzing the basic statistics of the waiting days:

In [ ]:
get_statistics(dataset.WaitingDays)

The **average waiting is about 10 days**, with a standard deviation of approximately 15 days. These values suggest that this distribution is not normal, since the mean is not in the center, as confirmed by its histogram. 

Looking back at the *waitingdays* dataframe created in <a href="#sec2.3.3">Section 2.3.3</a>, we can compare the no-showing rate (i.e. the number of patients that missed the appointment against those who attended) in relation to each of *WaitingDays* categories.

In [ ]:
## Showing the data again:
waitingdays

This data could be better understood graphically. To do so, we will first manipulate the *waitingdays* dataframe in
order to get the percentual information. We will also multiply by *500* the *No-showing rate* values in order to put this attribute in the same scale of the chart. For last, we will show all these information in the same chart, based on the codes proposed in <a href="#[6]">[6]</a> and <a href="#[7]">[7]</a>.

In [ ]:
## Adjusting the dataframe:
eda_waitingDays = waitingdays.copy()  #Copying the dataframe from Section 2.3.3
eda_waitingDays.reset_index(drop=False, inplace=True)  #Making the index as a column in order to be plotted.
eda_waitingDays.drop(7, inplace=True)  #Droppping the last row, since it's empty.

## Adding new columns:
#Transforming the 'No-showing rate' into strings with the percentual values:
eda_waitingDays['No-show percentual'] = eda_waitingDays['No-showing rate'].apply(lambda x: '{0:.2f}%'.format(x))
#Multiplying the rate values by 500 times in order to be plotted in the same scale:
eda_waitingDays['No-showing rate (500x)'] = eda_waitingDays['No-showing rate']*500

## Showing the adjusting dataframe:
eda_waitingDays

In [ ]:
## Setting the graph parameters:
fig1, ax = plt.subplots(figsize=[12,6])  #Defines the graph window size
fig1.subplots_adjust(top=0.92)
plt.suptitle('Appointments distribution by waiting time categories', fontsize=14, fontweight='bold')

colors = ['tab:blue', 'tab:green', 'tab:red']  #Defines the colors to be used

ax.set_ylabel('Number of occurences', color=colors[0], fontsize=12)  #Set the y-axis color and label
ax.tick_params(axis='y', labelcolor=colors[0])

## Plotting the line chart:
eda_waitingDays[['WaitingDays', 'No-showing rate (500x)']].plot(x='WaitingDays', linestyle='-', marker='o', ax=ax, color=colors[2])
#Setting the line chart marker labels
x = ax.get_xticks()  #Getting the x-axis ticks to plot the label
for a,b,c in zip(x,eda_waitingDays['No-showing rate (500x)'], eda_waitingDays['No-show percentual']):
    plt.text(a,b+1500,c, color='red', fontsize=14)
    
## Plotting the bar chart:
eda_waitingDays[['WaitingDays', 'No', 'Yes']].plot(x='WaitingDays', kind='bar', ax=ax, color=colors[0:2])

ax.set_xlabel('Waiting time categories', fontsize=12)  #Set the y-axis color and label

plt.show()

Through the chart above, it becomes evident that **the *no-showing rate* increases as the waiting gets longer**. It reaches the lower rates when the attendance occurs in the same day it was scheduled. We can then associate it with the emergency care or even when the patient goes to the health unit without previous scheduling.  

It is curious to notice that after one quarter waiting there is a slight return to the patient assiduity, when the *no-showing rate* downs from *49.6%* to *%34.78*. This question will be explored deeply in the next section.

<a id='sec3.1.2'></a>
#### 3.1.2. A return to assiduity
As noted in the chart above, the *no-showing rate* increases along the waiting time, however it starts to decrease after one quarter waiting. To get insights for the next analysis (the patient profiles will be explored in <a href="EDA_question2">Section 3.2</a>), we will now compare some basic statistics of the numerical attributes from the group patients attending to the appointments in the same day **(Group I)** and after one quarter **(Group II)**. 

In [ ]:
## Group I - Describing the numerical attributes for the same day appointments:
group_I = dataset[dataset['WaitingCategories'] == 'Same day: 0'].describe()
group_I

In [ ]:
## Group II - Describing the numerical attributes for the semester appointments:
group_II = dataset[dataset['WaitingDays']>90].describe()
group_II

Despite the number of instances are in a proportion of 1:217, the basic statistics from these two groups of patients reveals interesting characteristics we can therefore associate to the reduction of the no-showing rate. To help finding them, we will make use of the *find_differences()* function, as defined below, applied to the *mean* values of each group.

In [ ]:
def find_differences(serie1, serie2, pct_diff):
    '''Given two data series [serie1, serie2], compare those attributes and return 
    those who difference among them is higher than pct_diff (e.g. 50% must be entered as 0.5).
    The index of both series must be identical.
    '''
    try:
        if (serie1.index.all() == serie2.index.all()):
            ## Calculating the differences
            testA = serie1 / serie2
            testB = serie2 / serie1
            checkA = [x for x in testA if (x > pct_diff)&(x<1)]
            checkB = [x for x in testB if (x > pct_diff)&(x<1)]
            
            ## Showing which attributes in serie1 are less than "pct_diff" of those in serie1:
            print('Attributes in "Serie I" whose values are less than {0:.1f}% of those in "Serie II":'.format(pct_diff*100))
            for item in checkA:
                print('\t{0}: {1:.1f}%'.format(testA[testA == item].index[0], item*100))
            
            ## Showing which attributes of serie2 are "pct_diff" higher in serie1:
            print('Attributes in "Serie II" whose values are less than {0:.1f}% of those in "Serie I":'.format(pct_diff*100))
            for item in checkB:
                print('\t{0}: {1:.1f}%'.format(testB[testB == item].index[0], item*100))
    except ValueError:
        print('The series must have same index and length!')
    return    

In [ ]:
find_differences(group_I.loc['mean'], group_II.loc['mean'], 0.30)

Based on the findings above, the both group of patients can be characterized as follow, considering those attributes with a difference in the mean values higher than 30% of the other group:

* Group I - Same day scheduling appointment:
  * Average age: 34.5 years old (young adults)
  * Scholarship: **10% receives scholarship**
  * Alcoholism: present in **3.9% of the group individuals**
  * Hipertension: present in 17.5% of the group individuals
  * Diabetes: present in 6.6% of the group individuals
  * Handicap: present in 2.4% of the group individuals
  
* Group II - A semester waiting between schedulling and the appointment:
  * Average age: **60.8 years old (senior)**
  * Scholarship: 6.4% receives scholarship
  * Alcoholism: present in 1.4% of the group individuals
  * Hipertension: present in **53.9% of the group individuals**
  * Diabetes: present in **13.4% of the group individuals**
  * Handicap: present in **6.9% of the group individuals**  
  
From this analysis, one hipothesys for the *no-showing rate* reduction for waiting periods higher than a quarter can be assigned to the patient profile: **elderly people as well as those with chronic diseases who require regular medical follow-up tend to schedule long-term appointments and attend to them.**

<a id='sec3.1.3'></a>
#### 3.1.3 Neighborhood analysis
This last analysis seeks to find out how the *no-showing appointments* are distributed among the health units. In other words, how many appointments are registered to each health unit and if they were attended to or not. To find this out, we will carry two slightly distinct analysis based on the neighborhood attribute: 
* (i) How the absence number is distributed along the neighborhoods? 
* (ii) We have already seen that the no-showing rate increases as the waiting gets longer. Is the validity of this statement indifferent to geographical location?

To find this out, we will first group the dataset by the neighborhood names, followed by some data manipulation to gather the desired information for each question.

#### (i) No-showing rate by neighborhood
In this analysis we will show the absence number distribution into two ways: first in absolute numbers, to get not only the information of which health unit presents more absence, but to know which one has the highest number of attendments. Second, it is interesting to compare the normalized data, i.e. how much the absence of each health unit represents the total of its attendments.

In [ ]:
## Using the pandas.groupby() method to generate a pivot table:
neighbors_I = dataset.groupby(by='Neighbourhood').No_show.value_counts().sort_index()

In [ ]:
## Manipulating the data:
neighbors_I = neighbors_I.unstack()  #Converting the groupby object into a dataset
neighbors_I.fillna(value=0, inplace=True)  #Replacing NaN values by zero
print(neighbors_I.head(3))

In [ ]:
## Normalizing the data using a predefined function:
normalNeighbor = df_row_normalize(neighbors_I)
print(normalNeighbor.head(3))

In [ ]:
## Getting the normalized data statistics:
normalNeighbor.describe()

In [ ]:
## Adding a total column:
neighbors_I['Total'] = get_total(neighbors_I)
normalNeighbor['Total'] = get_total(normalNeighbor)

In [ ]:
#Reseting the 'neighbourhood' index and making it as a column:
neighbors_I.reset_index(inplace=True)  
normalNeighbor.reset_index(inplace=True)

In [ ]:
## Initialize the matplotlib figure:
fig2, (ax1, ax2) = plt.subplots(1,2, figsize=(12,16), sharey=False)
fig2.tight_layout()  #When working with 'tight_layout', the subplot must be adjusted [https://stackoverflow.com/questions/7066121/how-to-set-a-single-main-title-above-all-the-subplots-with-pyplot]
fig2.subplots_adjust(top=0.96)  #Adjusting the space for the superior title

## Plot the relative absence by neighborhood
#Total appointments
sns.set_color_codes("pastel")
sns.barplot(x="Total", y="Neighbourhood", data=normalNeighbor, label="Total", color="b", ax=ax1)
#Attended appointments
sns.set_color_codes("muted")
sns.barplot(x="No", y="Neighbourhood", data=normalNeighbor, label="Attended", color="b", ax=ax1)
## Add a legend and informative axis label
ax1.legend(ncol=2, loc="lower left", frameon=True)
ax1.set(xlim=(0, 1), ylabel="", xlabel="Relative attended appointments by neighborhood")
sns.despine(left=True, bottom=True,ax=ax1)

## Plot the absolute absence by neighborhood
#Total appointments
sns.set_color_codes("pastel")
sns.barplot(x="Total", y="Neighbourhood", data=neighbors_I, label="Total", color="b",ax=ax2)
#Attended appointments
sns.set_color_codes("muted")
sns.barplot(x="No", y="Neighbourhood", data=neighbors_I, label="Attended", color="b", ax=ax2)
## Add a legend and informative axis label
ax2.legend(ncol=2, loc="lower right", frameon=True)
ax2.set(xlim=(0, 7720), ylabel="", xlabel="Absolute attended appointments by neighborhood")  #The xlim value comes from the maximum value in the dataset.
ax2.set_yticklabels([''])
sns.despine(left=True, bottom=True, ax=ax2)

plt.suptitle('Attended appointments by neighborhood', fontsize=14, fontweight='bold')
plt.show()

From the analyzed data we found out that 80% of the appointments are attended to, with a standard deviation of 3.8%. The distribution of this numbers are easily perceived on the charts above. The chart representing the absolute values (the right one) is important to avoid biased interpretations: the neighborhood *Parque Industrial* had attended all its appointments but the same neighborhood carries the lowest number of appointments.

#### (ii) Relative absence: neighborhoods versus waiting categories
The main purpose of this analysis is to check if the no-showing rate increases as the waiting gets longer along all the considered geographical locations. In order to get to this we will make use of a *heatmap*, which in turn is an array where the columns represent one attribute and the rows represent another. The value of each cell represents the correlation between these attributes and, for ease of viewing, is converted to a color map <a href="#[9]">[9]</a>. In our case, the columns represent the *waiting time categories* while the rows represent the *neighborhoods*. The value (or color) of each cell represents the percentage of no-showing appointments.

In [ ]:
## Using the pandas.groupby() method to produce a pivot table:
neighbors_II = dataset.groupby(by=['Neighbourhood','No_show']).WaitingCategories.value_counts().sort_index()

In [ ]:
## Manipulating the data:
neighbors_II = neighbors_II.unstack(1).unstack()  #Converting the groupby object into a dataset
neighbors_II.fillna(value=0, inplace=True)  #Replacing NaN values by zero
neighbors_II = df_row_normalize(neighbors_II)  #Normalizing its values by the total of each row
neighbors_II = neighbors_II['Yes']  #Keeping only the values related to the absence
## Converting the normalized float values to percentual int values:
neighbors_II = (neighbors_II*100).astype('int64')
neighbors_II = neighbors_II.reindex(columns=['Same day: 0', 'Short: 1-3', 'Week: 4-7', 'Fortnight: 8-15', 'Month: 16-30', 'Quarter: 31-90', 'Semester: 91-180'])

In [ ]:
# Drawing a heatmap with the numeric values in each cell
fig3, ax = plt.subplots(figsize=(10, 25))
fig3.subplots_adjust(top=.965)
plt.suptitle('Relative absence distributed by neighborhood and waiting categories', fontsize=14, fontweight='bold')

cbar_kws = {'orientation':"horizontal", 'pad':0.08, 'aspect':50}
sns.heatmap(neighbors_II, annot=True, fmt='d', linewidths=.3, ax=ax, cmap='RdPu', cbar_kws=cbar_kws);

It can be drawn from the heatmap above that in general the no-showing rate increases as the waiting gets longer for most of neighborhood, besides slightly different for each local. Some neighborhoods have higher no-showing rates even for short waiting times (e.g. *Ilha do Frade* and *Morada de Camburi*).  

From these insights, it would be reasonable for the public administration of these localities to investigate the reasons for such occurrences by collecting more detailed data and conducting deeper analysis.

<a id='EDA_question2'></a>
### 3.2. Exploring the patient profiles
After understanding the characteristics of the missed appointments, as explored in the last section, we will now analyze the patient profiles driven by the following questions:
* How the *no-shows* are distributed among the patients? 
* Is there any common characteristics among those patients that miss appointments?
* Patients suffering from serious illnesses are more assiduous?

Since we are now interested in the patients attribute that could characterize and differentiate those who attends to appointments from those who does not, we will first manipulate the dataset to show only the relevant information.

In [ ]:
## Defining a new dataframe from the attributes of interest:
patients = dataset[['Gender','Age','Scholarship','Hipertension','Diabetes',
                    'Alcoholism','Handicap','WaitingCategories','SMS_received','No_show']]

By using *pandas.groupby()* method we could extract the relation among *waiting categories* and the patient attributes like *age, hipertension, diabetes* and so forth, primarily separated among the *No_show* classes. This task is easily accomplished by the *sum()* method, except for the *Age* attribute which must be calculated by the *mean()* method.

In [ ]:
## Obtaining an statistical overview of all the attributes:
patients.groupby(by=['No_show','WaitingCategories']).describe()

In [ ]:
## Grouping by classes and waiting categories and calculating the instances sum:
patients_sum = patients.groupby(by=['No_show','WaitingCategories']).sum()
## Grouping by classes and waiting categories and calculating the instances sum:
patients_mean = patients.groupby(by=['No_show','WaitingCategories']).mean()

In [ ]:
## Adjusting the 'Age' attribute to have the mean instead of sum values:
patients = patients_sum.copy()
patients['Age'] = patients_mean['Age']

In [ ]:
## Normalizing data using the predefined function
patients = df_column_normalize(patients, percent=True)

In [ ]:
# Drawing a heatmap with the numeric values in each cell
fig4, ax = plt.subplots(figsize=(12, 10))
fig4.subplots_adjust(top=.94)
plt.suptitle('Distribution of patients attributes by waiting categories and no-showing classes', fontsize=14, fontweight='bold')

ax.set_yticklabels(ax.get_yticklabels(), ha="right", fontsize=12, weight='bold')
ax.set_xticklabels(ax.get_xticklabels(), fontsize=12, weight='bold')

cbar_kws = {'orientation':"horizontal", 'pad':0.05, 'aspect':50}
sns.heatmap(patients, annot=True, fmt='.2f', linewidths=.3, ax=ax, cmap='RdPu', cbar_kws=cbar_kws);

From the heatmap and descriptive statistics above the following conclusions can be drawn:
* The patients who attend to the appointments are in general older than those who don't.
* Most of the patients **who attend** the appointments scheduled in the **same day** receives *scholarship* and presents *hipertension, diabetes, alcoholism*, and *handicap*. These data may indicate these group of patients (i) may not have access to scheduling systems; or (ii) may need emergency care more often.
* Patients suffering from hypertension and diabetes are more assiduous in relation to medical appointments. 
* Most **patients who have received SMS (70%) have attended to appointments scheduled from a week to a quarter**.

-----------

<a id='conclusions'></a>
## 4. Conclusions
This analysis sought to explore the medical appointment no-shows dataset, available in <a href="#[2]">[2]</a>, containing more than *100.000* appointments and of which approximately 30% of the patients have not attended. In this way, the data exploration presented in this notebook had the purpose to show up some insights on the possible causes of this appointment missing. It is important to notice, though, that with the provided data and the applied methods it is not possible to affirm any statement with statistical validity nor causal relations. On the other hand, the insights obtained through this analysis can, instead, inform and direct in-depth research on the subject, in order to validate the causes of the appointment missing and find ways to eradicate them.

----------
<a id='sec4.1'></a>
### 4.1. Analysis limitations

This section aims to expose some limitations and challenges faced during the data analysis process in order to help improving the data for further analysis:

* **Concerning data acquisition**:  
The dataset description affirms it comes from public health units from a specific municipality in Brazil. There are some data issues, though, that should be clarified: 
    * There is no description on how the data were acquired;
    * There is no description on how the patients ID were anonymized;
    * It is not known if the health units share the same patient database. From this it follows that is not possible to know if the same patient receives different identification codes when he/she goes to a different health unit.  
    

* **Concerning the data scope**:  
There are some references in the data source to the Brazilian health system (SUS) although the provided data are only related to municipal health units from *Vitória, ES*. As exposed in Section 1.2, the SUS is very heterogeneous and concerns to a country with almost 200 million people. In this way, it is not possible to infer the no-showing causes or to generalize any rules from this data.  
Besides that, there are some inconsistencies regarding the *Neighborhood* attribute. In the data description and in some forum comments it is said this attribute refers to the health unit location. However it is late stated it comes from 45 health units (against 80 unique neighborhood names). For last, it should be provided more details about the health unit and patients working and residence addresses, which in turn could allow in-depth analysis associating the displacement habits of each patient as well as the socioeconomic characteristics of each neighborhood. 


* **Concerning the data quality**:
    * Besides there were few data to be cleaned in the provided dataset, some of them suggest a process issue that should be verified in the case of an in-depth research. To cite some identified in this analysis: *negative age*; *scheduling date older than appointment date*.


* **Concerning the data analysis**:
Given the above, the present analysis was limited to categorizing the patient waiting time and exploring the associations of these categories with other attributes. Nevertheless, it was possible to provide relevant insights that could lead to a controlled data acquisition for future researches on this topic.

-------------
<a id='sec4.2'></a>
### 4.2. Analysis overview

In the <a href="#intro">Introduction</a> section of this notebook we have formulated some questions that have driven this analysis. We will now present the insights gained from the data exploration for each of them.

* **Regarding the no-showing appointments**:
  - Do the patients forget the appointment?
  - What is the average waiting time between the scheduling date and the appointment date?

<font color='#21618C'> From the chart below it can be seen that probably people forget the appointment, since **the no-showing rate increases as the waiting time gets longer**. As it can be seen by the red line in the chart, the no-showing rate reaches its lower value when the attendance occurs in the same day it was scheduled. We can then associate it with the emergency care or even when the patient goes to the health unit without a previously scheduling.<br>    

It is curious to notice that after one quarter waiting there is a slight return to the patient assiduity, when the *no-showing rate* downs from *49.6%* to *%34.78*. From the analysis in Section 3.1.2, a possible hipothesys for the *no-showing rate* reduction in waiting periods higher than a quarter can be assigned to the patient profile: **elderly people as well as those with chronic diseases who require regular medical follow-up tend to schedule long-term appointments and attend to them.**<br>  

For last, considering the whole dataset the average waiting is of *10.18 days* with a standard deviation of *15.25 days*.
</font>


In [ ]:
fig1 ##This chart was generated in Section 3.1.1

--------------

* **Regarding the patient profile**:
  - Is there any common characteristics among those patients that miss appointments?
  - What is the *missing appointments per patient* relation in the dataset?  
  - How many patients have missed an appointment at least once?
  - Patients suffering from serious illnesses are more assiduous?

<font color='#21618C'> The possible answers for these questions were explored at Section 3.2 and comes from descriptive statistics, as those shown in the heatmap below. The following conclusions could be drawn:  
<ul>
<li>* The patients who attend to the appointments are in general older than those who don't.
<li>* Most of the patients **who attend** the appointments scheduled in the **same day** receives *scholarship* and presents *hipertension, diabetes, alcoholism*, and *handicap*. These data may indicate these group of patients (i) may not have access to scheduling systems; or (ii) may need emergency care more often.
<li>* Patients suffering from hypertension and diabetes are more assiduous in relation to medical appointments. 
<li>* Most **patients who have received SMS (70%) have attended to appointments scheduled from a week to a quarter**.
</ul>
</font>


In [ ]:
fig4 ##This chart was generated in Section 3.2

--------------

* **Regarding the health unit**:
  - What are the neighbourhoods with the highest no-showing rate?
  - Considering there are common characteristics among those who do not attend the appointments, how these characteristics are geographically distributed?
  

<font color='#21618C'> These questions are directly answered by the three charts below. From the analyzed data we found out that 80% of the appointments are attended to, with a standard deviation of 3.8%, and the geographical  distribution of this number is easily perceived on the bar charts below. The neighborhood with the highest relative no-showing rate is *Santos Dumont*.<br>  

From the heatmap it can be drawn that in general the no-showing rate increases as the waiting gets longer for most of neighborhood, besides slightly different for each local. Some neighborhoods have higher no-showing rates even for short waiting times (e.g. *Ilha do Frade* and *Morada de Camburi*).<br>  

From these insights, it would be reasonable for the public administration of these localities to investigate the reasons for such occurrences by collecting more detailed data and conducting deeper analysis.
</font>

In [ ]:
fig2 ##This chart was generated in Section 3.1.3

In [ ]:
fig3 ##This chart was generated in Section 3.1.3

--------------

* **Regarding the data**:
  - Is the data balanced in relation to the interest class (showing/no-showing to appointments)?
  - How is the data distributed in relation to the location (neighborhood)? I.e. all the units have an equivalent and balanced number of instances?
  


<font color='#21618C'> As it was identified in Sections 2.3.1 and 2.3.5, the data are unbalanced in relation to *gender* attribute, as well as to the *No_showing* class. From the *absolute attended appointments bar chart* above it is also clear the data are unbalanced in relation to the health units. 

--------------

<a id='references'></a>
## References

<a id='[1]'></a>[1] Wikipedia contributors. Brazil. Wikipedia, The Free Encyclopedia. February 9, 2018, 22:06 UTC. Available at: https://en.wikipedia.org/w/index.php?title=Brazil&oldid=824851232. Accessed in February 10, 2018.

<a id='[2]'></a>[2] Kaggle. *Medical Appointment No Shows*. Reviewed dataset. Available at: https://www.kaggle.com/joniarroba/noshowappointments. Accessed in February 10, 2018.

<a id='[3]'></a>[3] Victora, C. G., Barreto, M. L., Leal, M. D. C., Monteiro, C. A., Schmidt, M. I., Paim, J. S., ... & Reichenheim, M. E. (2011). *Condições de saúde e inovações nas políticas de saúde no Brasil: o caminho a percorrer* (in Portuguese). Available at: https://repositorio.ufba.br/ri/handle/ri/5973. Accessed in February 12, 2018.

<a id='[4]'></a>[4] Pandas 0.22.0 documentation. Available at: https://pandas.pydata.org/pandas-docs/stable/timeseries.html. Accessed in February 20, 2018.

<a id='[5]'></a>[5] Twelve useful Pandas techniques in Python for data manipulation. Analytics Vhidya. Available at: https://www.analyticsvidhya.com/blog/2016/01/12-pandas-techniques-python-data-manipulation/. Accessed in February 22, 2018.

<a id='[6]'></a>[6] Stack Overflow. Plot pandas DataFrame as bar and line on the same one chart. Available at https://stackoverflow.com/questions/23482201/plot-pandas-dataframe-as-bar-and-line-on-the-same-one-chart. Accessed in February 24, 2018.

<a id='[7]'></a>[7] Stack Overflow. Writing numerical numerical values on the plot with Matplotlib. Available at https://stackoverflow.com/questions/6282058/writing-numerical-values-on-the-plot-with-matplotlib. Accessed in February 24, 2018.

<a id='[8]'></a>[8] Wikipedia contributors. *Lista de Bairros de Vitória*. Available at: https://pt.wikipedia.org/wiki/Lista_de_bairros_de_Vit%C3%B3ria. Accessed in February 27, 2018.

<a id='[9]'></a>[9] Matplotlib. Colorbar API. Version 2.1.2. Available at https://matplotlib.org/api/colorbar_api.html. Accessed in February 27, 2018.